## Model 5: IF + FS1

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/iforest_df_test_cleaned_pred.csv',sep=';', index_col=0)

In [3]:
df['iforest_score'] = -1 * df['iforest_score']

A score close to 1 indicates anomalies.  
Score much smaller than 0.5 indicates normal observations.  
If all scores are close to 0.5 then the entire sample does not seem to have clearly distinct anomalies.  

In [4]:
df[df['iforest_score'] < 0]

,iforest_score,ind_var30,var36,var15,num_var4,num_var8_0,ind_var13_0,ind_var12_0,saldo_var30,ind_var39_0,...,num_op_var40_hace2,saldo_var20,num_sal_var16_ult1,delta_imp_reemb_var17_1y3,saldo_medio_var33_hace3,num_op_var40_comer_ult3,num_var29_0,num_aport_var33_ult1,TARGET,ID
21956,-0.011330,1,3,27,1,0,0,0,60.0,1,...,0,0.0,0,0,0.0,0,0,0,0,43966
18494,-0.011330,1,3,23,1,0,0,0,3.0,1,...,0,0.0,0,0,0.0,0,0,0,0,37070
64217,-0.011330,1,3,23,1,0,0,0,3.0,1,...,0,0.0,0,0,0.0,0,0,0,0,128332
64974,-0.006915,1,99,23,1,0,0,0,15.0,1,...,0,0.0,0,0,0.0,0,0,0,0,129825
40704,-0.011330,1,3,34,1,0,0,0,3.0,1,...,0,0.0,0,0,0.0,0,0,0,0,81438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,-0.011330,1,3,30,1,0,0,0,90.0,1,...,0,0.0,0,0,0.0,0,0,0,0,34778
58379,-0.008210,0,99,25,0,0,0,0,0.0,1,...,0,0.0,0,0,0.0,0,0,0,0,116511
14402,-0.011330,1,3,23,1,0,0,0,3.0,1,...,0,0.0,0,0,0.0,0,0,0,0,28954
10520,-0.006388,0,99,53,0,0,0,0,0.0,1,...,0,0.0,0,0,0.0,0,0,0,1,21097


prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [5]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket']  = pd.qcut(data[prob].rank(method='first'), 5)
    # data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [6]:
from scipy.stats import ks_2samp
yhat = df['iforest_score']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.22754297530613538

In [7]:
results_quintil = ks(df, target='TARGET', prob='iforest_score')
results_quintil

KS is 12.0% at decile 3


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.005195,0.301244,126,2915,21.0,20.0,21.0,20.0,0.8
2,-0.005085,0.005195,183,2858,30.0,20.0,51.0,40.0,11.4
3,-0.008210,-0.005085,125,2915,21.0,20.0,71.0,60.0,12.0
4,-0.011330,-0.011330,43,2998,7.1,21.0,79.0,80.0,-1.5
5,-0.011330,-0.008210,130,2911,21.0,20.0,100.0,100.0,0.0


In [8]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['iforest_score']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['iforest_score']))-1)

Roc Auc:  0.5920072944960932
Gini:  0.18401458899218648


In [9]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model5_iforest_fs1.csv",sep=';')